In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import requests
response = requests.get("https://huggingface.co")
print(response.status_code)  # Should print 200

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
from huggingface_hub import login
login(token=hf_token)

In [ ]:
!pip install -q transformers peft accelerate bitsandbytes datasets

In [ ]:
!nvidia-smi

In [ ]:
import pandas as pd
from datasets import Dataset

# Load data
df = pd.read_excel("/kaggle/input/aiscam-dataset/SyntheticDataSet_Call_Text_5.xlsx", engine="openpyxl")

# Clean data
df = df[['text', 'label']].dropna()
df['label'] = df['label'].str.strip().str.lower().map({'fraud': 'fraud', 'normal': 'normal'})

# Prompt-format the dataset
df['prompt'] = df['text'].apply(lambda x: f"Classify this call:\n{x}\nLabel:")
df['response'] = df['label']

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['prompt', 'response']], preserve_index=False)

# Train-test split
dataset = dataset.train_test_split(test_size=0.1)


In [ ]:
import shutil
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# Copy Gemma model to writable path
src_path = "/kaggle/input/gemma/transformers/default/1/gemma-2b"
dst_path = "/kaggle/working/gemma-2b"
if not os.path.exists(dst_path):
    shutil.copytree(src_path, dst_path)

# Load model + tokenizer
tokenizer = AutoTokenizer.from_pretrained(dst_path, use_fast=True, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(dst_path, torch_dtype=torch.float16, device_map="auto", local_files_only=True)


In [ ]:
def tokenize(example):
    return tokenizer(
        example['prompt'],
        text_target=example['response'],
        truncation=True,
        padding="max_length",
        max_length=256
    )

tokenized = dataset.map(tokenize, batched=True)


In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


In [ ]:
import time
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="/kaggle/working/gemma2b-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    save_total_limit=1,
    fp16=True,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

start = time.time()
print("Training started...")
trainer.train()
print(f"✅ Training completed in {(time.time()-start)/60:.2f} minutes")


!ps -ef 

!nvidia-smi


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load fine-tuned model from saved directory
model_dir = "/kaggle/working/gemma2b-finetuned"

tokenizer = AutoTokenizer.from_pretrained(model_dir, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_dir, local_files_only=True)
model.eval()

# Auto-select device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
def classify(text):
    prompt = f"Classify this call:\n{text}\nLabel:"
    
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate model output
    output = model.generate(
        **inputs,
        max_new_tokens=5,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and extract label
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    label = decoded.split("Label:")[-1].strip().split()[0].lower()

    if "fraud" in label:
        return "fraud"
    elif "normal" in label or "legit" in label:
        return "normal"
    else:
        return f"Uncertain: {label}"


!zip -r gemma2b-finetuned.zip /kaggle/working/gemma2b-finetuned

from IPython.display import FileLink
FileLink(r'gemma2b-finetuned.zip')